<a href="https://colab.research.google.com/github/ftmthb/NER/blob/main/ner_india_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The notebook flow

* **Environment set**:
This consisits of:
  *  creating a virtual environment in which we'll install necessary libraries in order to avoid re-installing them in each session.
  * With that being done, we'll only need to append the path of the installation in the virtual environment at the begining, which is somehow the equivalent to the activation

* **Preprocess**: The goal of this part is to <ins> create training and development datasets </ins>. In order to achieve that we'll:
  * import the json data: train and developpement ( validation)
  * get the indexes of the documents <ins>with annotations</ins>, in other terms with special entities
  * clean the data: this function aims to get rid of all the special characters that are either ignored or in proper terms considered as whitespaces by the tokenizer or are going to be a waste ( in some documents we have a successsion of '===', '***', to separate passages, each element of these will be tokenized as one element which would be better avoided since we only have 512 tokens). On the other hand, since the whitespaces will be removed by the tokenizer, we don't really need to remove them in the cleaning proccess.
  * define a function to get the new position of a special entity based on its position in the text given in the annotations because the cleaning will imply some changes in the positions and we'll need those later to make sure the labels will be aligned.
  * define the model, the tokenizer and tokenize the data. The tokenization will result on a list of tokens for each document.
  * define a function that'll get the position of the token of a given special entity based on its position in the text. This function will be used to make sure the labels will be aligned with the tokens.
  
  * create the list of labels' tokens based on the list of tokens. (Considering that the tokens are a list of lists. A list of tokens for each document where each document is an element of the bigger list)
  *  gather all the labels: labels_list, which will be encoded
  * create the datasets:
    * with 0 for cls, sep and pad
    * with -100 for cls, sep and pad
* **Process**:
  * load the datasets
  * fine tune model on different datasets


# Conclusion


* The model shows better performance when we don't affect -100 on cls, sep and pad
* Some labels (GPE, OTHER_PERSON) gives the following warning

    &emsp; | seems not to be NE tag   \\
after checking, these labels are present following the IOB/BIO format (we have B-GPE, I-GPE, B-OTHER_PERSON and I-OTHER_PERSON ammoung the labels) and raw (we also have GPE and OTHER_PERSON)
which could be the source of the confusion


# 1.Environment set

## Create and activate venv to avoid installing libraries in every session

In [ ]:
!pip install virtualenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 15.9 MB/s eta 0:00:00


In [ ]:
!virtualenv /content/drive/MyDrive/virtual_env


created virtual environment CPython3.10.12.final.0-64 in 42682ms
  creator CPython3Posix(dest=/content/drive/MyDrive/virtual_env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: MarkupSafe==2.1.5, PyYAML==6.0.1, accelerate==0.30.1, aiohttp==3.9.5, aiosignal==1.3.1, async_timeout==4.0.3, attrs==23.2.0, certifi==2024.2.2, charset_normalizer==3.3.2, datasets==2.19.1, dill==0.3.8, filelock==3.14.0, frozenlist==1.4.1, fsspec==2024.3.1, huggingface_hub==0.23.0, idna==3.7, jinja2==3.1.4, langdetect==1.0.9, mpmath==1.3.0, multidict==6.0.5, multiprocess==0.70.16, networkx==3.3, numpy==1.26.4, nvidia_cublas_cu12==12.1.3.1, nvidia_cuda_cupti_cu12==12.1.105, nvidia_cuda_nvrtc_cu12==12.1.105, nvidia_cuda_runtime_cu12==12.1.105, nvidia_cudnn_cu12==8.9.2.26, nvidia_cufft_cu12==11.0.2.54, nvidia_curand_cu12==10.3.2.106, nvidia_cusolver_cu12==1

In [ ]:
import sys
sys.path.append("/content/drive/MyDrive/virtual_env/lib/python3.10/site-packages")

### install libraries

#### datasets

In [ ]:
!source /content/drive/MyDrive/virtual_env/bin/activate; pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.0 MB/s eta 0:00:00
  Using cached packaging-24.0-py3-none-any.whl.metadata (3.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 19.1 MB/s eta 0:00:00
Using cached packaging-24.0-py3-none-any.whl (53 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 4.7 MB/s eta 0:00:00
   ━━━

#### transformers [torch]

In [ ]:
!source /content/drive/MyDrive/virtual_env/bin/activate; pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.1/774.1 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

#### accelerate

In [ ]:
!source /content/drive/MyDrive/virtual_env/bin/activate; pip install accelerate -U

#### seqeval

In [ ]:
!source /content/drive/MyDrive/virtual_env/bin/activate; pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 950.6 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=911957a1f4c77fac1770f1f20b38337d07b0c706572a68b5950cc7c9f4c6ac67
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


#### evaluate

In [ ]:
!source /content/drive/MyDrive/virtual_env/bin/activate; pip install evaluate

# 2.Preprocess

#### append path
 (equiv to activate the virtualenv at the begining of fevery session, to avoid reinstalling libraries)
 (only run if the installation was done in a different previous session, in other terms check if the appended path is not already in sys.

In [ ]:
import sys
sys.path.append("/content/drive/MyDrive/virtual_env/lib/python3.10/site-packages")

### Data import
+only taking dcts with annotations, clean,

In [ ]:
import json

with open(r"/content/drive/MyDrive/ner_india/NER_TRAIN_JUDGEMENT.json") as file:
    json_data = json.load(file)
len(json_data)

9435

In [ ]:
import json
with open(r"/content/drive/MyDrive/ner_india/NER_DEV_PREAMBLE.json") as file:
    dev_data = json.load(file)
print(len(dev_data))

125


#### only taking the indices with annotations

In [ ]:
with_annot_indices = []
for indice in range(len(json_data)):
  for annot in json_data[indice]['annotations']:
    if annot['result']!= []:
      with_annot_indices.append(indice)

print('num of dcts with no annot:', len(json_data) - len(with_annot_indices))

num of dcts with no annot: 2177


#### clean the text data

Technically, there is no need to get rid of whitespaces, because if the goal is really to prepare the data for the tokenizer, they aren't even taken into consideration. We only need to get rid of the special characters, such as \xad, \xa0, etc..., (they could be found using re.finditer(r'[\x80-\xff]', text), to find all possible itterations). To make sure we cover all the special characters, we need to find all the posibilities inside the text data

In [ ]:
import re
def clean_text_data(text_data):
    replacements = [
        (r'\.{2,}', ' '),
        (r'\={2,}', ' '),
        (r'\-{2,}', ' '),
        (r'\xad', ' '),
        (r'\xa0', ' '),
        (r'\x80', ''),
        (r'\x9d', ''),
        (r'\x13', ' '),
        ]

    for old, new in replacements:
      text_data = re.sub(old, new, text_data)
    return text_data

dev_text_data = []
for line in dev_data:
    dev_text_data.append(clean_text_data(line['data']['text']))

text_data = []
plain_text_data = []
# for line in json_data:
for idx in with_annot_indices:
    text_data.append(clean_text_data(json_data[idx]['data']['text']))
    plain_text_data.append(json_data[idx]['data']['text'])

#### get new position in clean data function

In [ ]:
def get_new_position(old_text, new_text, pos):
  #a function that'll allow to find the position of a special entity in the new cleaned text using the position in the raw text given in annotations

  if new_text[pos] == old_text[pos]: #pos did not change
    return pos

  else: # we actually need to update the pos

    special_entity = old_text[pos]
    if clean_text_data(special_entity).split() == []:
      return 'not so special entity'

    if special_entity[0:2] == '--':
      special_entity = special_entity[2:] #cases where the NE starts with -, preceeded by -- in data['text'] and omitted by the cleaninh function
      pos = slice(pos.start + 2, pos.stop)

    if special_entity[0:1] == '-' and special_entity[1:2] != '-':
      special_entity = special_entity[1:] #cases where the NE starts with -, preceeded by -- in data['text'] and omitted by the cleaninh function
      pos = slice(pos.start + 1, pos.stop)

    in_old_text = [m.start() for m in re.finditer(re.escape(special_entity), old_text)]
    in_new_text = [m.start() for m in re.finditer(re.escape(clean_text_data(special_entity)), new_text)]

    if len(in_new_text) != len(in_old_text) and special_entity != '.':  #written differently #with whitespaces, or special characters
      # this block is for test, and shouldn't be achieved
      print(indice, "\n not same len")
      print('\t special_entity: ', special_entity, 'at pos', pos)
      print('in_old_text: ', in_old_text, '\t in_new_text: ',in_new_text)

    else:

      all_occ = dict(zip(in_old_text,in_new_text))

      start = all_occ[pos.start]

      end = start+len(clean_text_data(old_text[pos]))#-1

      # check
      if clean_text_data(old_text[pos]) != new_text[slice(start, end)]:
        print('not similar', old_text[pos], '\t', new_text[slice(start, end)])


      return slice(start,end)


### model and tokenizer
init, tokenize train and dev data,

In [ ]:
model = 'dslim/bert-base-NER'
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model)

#### tokenize train data

In [ ]:
tokenized = tokenizer(text_data,  is_split_into_words = False, return_offsets_mapping = True, add_special_tokens = True, truncation = True, padding = 'max_length', max_length=512, stride=128, return_overflowing_tokens=True,
 )  #when text is not splitted, the positions in str are accurate

tokens = [tokenizer.convert_ids_to_tokens(tokenizedx) for tokenizedx in tokenized.input_ids]

#### tokenization of the dev data with truncation and return overflowing

In [ ]:
dev_tokenized = tokenizer(dev_text_data,  is_split_into_words = False,  return_offsets_mapping = True, add_special_tokens = True,  padding = 'max_length', truncation = True, max_length=512, stride=128, return_overflowing_tokens=True, )  #when text is not splitted, the positions are accurate

dev_tokens = [tokenizer.convert_ids_to_tokens(tokenizedx) for tokenizedx in dev_tokenized.input_ids]

## Dataset preparation
get token pos, affect labels to train and dev

#### get token pos

In [ ]:
def get_token_pos(mappers, start_str, end_str):
  # get position in tokens list from annotations
  for idx, (start, end) in enumerate(mappers):
    if idx<= mappers.index(max(mappers)):
        if start <= start_str:
            idx_start = idx
        if end == end_str:
            idx_end = idx+1
        if end < end_str:
            idx_end = idx+2

  pos = slice(idx_start, idx_end)
  return pos

### Affectation

#### affect on train data

In [ ]:
#initialize all tokens labels as 'O': Outside
tokens_labels = [['O' for mapper in tokenized["offset_mapping"][i]] for i in range(len(tokenized["offset_mapping"]))]

for i, indice in enumerate(with_annot_indices):                #then update based on the str position of the special entity and the get token pos (from str to tokens)
  # print(indice)
  mappers = tokenized["offset_mapping"][i]
  raw_text = json_data[indice]['data']['text']

  for annot in json_data[indice]['annotations']:    #list of all the entities that are part of the deal
    for result in annot['result']:
      deb_str = result['value']['start']
      fin_str = result['value']['end']
      clean_pos = get_new_position(raw_text, text_data[i], slice(deb_str,fin_str) )
      if type(clean_pos) == slice:
        pos = get_token_pos(mappers, clean_pos.start, clean_pos.stop )

        label = result['value']['labels'][0]          #[0] to remove from list
        if tokens[i][pos][0] == result['value']['text']:   #when the special entity == token, not cut
          tokens_labels[i][pos.start] = label

        else:
          tokens_labels[i][pos.start] = 'B-'+label     #only put B- when there is an I-
          for x in range(pos.start+1, pos.stop):
            if x <len(tokens_labels[i]):
              tokens_labels[i][x] = 'I-'+label

        if ''.join(tokens[i][pos]).replace('##', '') != ''.join(text_data[i][clean_pos].split()):
          print(indice, i, pos,  text_data[i][clean_pos], pos, tokens[i][pos])

      else:
        print(clean_pos)



not so special entity
1423 1103 slice(40, 44, None) September  30,	 slice(40, 44, None) ['September', '30', ',', '1989']
3008 2335 slice(40, 48, None) Clause 8(vi)

 slice(40, 48, None) ['Claus', '##e', '8', '(', 'v', '##i', ')', '(']
3917 3038 slice(44, 53, None) section 3(1)(c)	  slice(44, 53, None) ['section', '3', '(', '1', ')', '(', 'c', ')', 'of']
4657 3606 slice(66, 74, None) Articles 14 and 19 (1)
 slice(66, 74, None) ['Articles', '14', 'and', '19', '(', '1', ')', '(']
5100 3940 slice(27, 35, None) Cr.P.C. 

  slice(27, 35, None) ['C', '##r', '.', 'P', '.', 'C', '.', 'C']
5824 4510 slice(13, 38, None) 'Paupuk Kannu Anni v. Thoppayya Mudaliar', (J) :   slice(13, 38, None) ["'", 'Pa', '##up', '##uk', 'Ka', '##nn', '##u', 'Ann', '##i', 'v', '.', 'T', '##hop', '##pa', '##yya', 'Mu', '##dal', '##iar', "'", ',', '(', 'J', ')', ':', 'Claus']
6042 4687 slice(34, 40, None) Rahmania Coffee Works. slice(34, 40, None) ['Rahman', '##ia', 'Coffee', 'Works', '.', '32']
6144 4778 slice(27, 29

#### affect on dev

In [ ]:
#the real deal with redundance

dev_tokens_labels = [['O' for mapper in dev_tokenized["offset_mapping"][i]] for i in range(len(dev_tokenized["offset_mapping"]))]

#the problem is that the affectation is done on the part at the end of the first chunk but not in the stride of the second chunk
#so i need to keep the new offset mapping with the positions to be able to get the index and affect
#problem solved


for indice in range(len(dev_data)):
  # print(indice)
  tkn_indice = dev_tokenized['overflow_to_sample_mapping'].index(indice)
  # print('tkn_indice: ', tkn_indice)

  for annot in dev_data[indice]['annotations']: #list of all the entities that are part of the deal
    for result in annot['result']:
      deb_str = result['value']['start']
      fin_str = result['value']['end']
      label = result['value']['labels'][0]

      raw_text = dev_data[indice]['data']['text']
      special_entity = result['value']['text']
      clean_pos = get_new_position(raw_text,dev_text_data[indice], slice(deb_str,fin_str) ) #will give the tupple for the offset_map

      if dev_tokenized['overflow_to_sample_mapping'].count(indice) > 1:

       #split into chunks: we need to check into both using offset mapping for example
        for i in range(dev_tokenized['overflow_to_sample_mapping'].count(indice)+1) :
          if clean_pos.start > dev_tokenized["offset_mapping"][tkn_indice+i+1][2][0] and clean_pos.stop < max(dev_tokenized["offset_mapping"][tkn_indice+i+1])[1]:

            mapper = dev_tokenized["offset_mapping"][tkn_indice+i+1]
            token_pos =  get_token_pos(mapper, clean_pos.start, clean_pos.stop)

            dev_tokens_labels[tkn_indice+i+1][token_pos.start] = 'B-'+label

            for x in range(token_pos.start+1, token_pos.stop):
              if x <len(dev_tokens_labels[tkn_indice+i+1]):
                dev_tokens_labels[tkn_indice+i+1][x] = 'I-'+label


          if clean_pos.start > dev_tokenized["offset_mapping"][tkn_indice+i][2][0] and clean_pos.stop < max(dev_tokenized["offset_mapping"][tkn_indice+i])[1]:
            #update the clean_pos too
            tkn_indice += i
            break

      mapper = dev_tokenized["offset_mapping"][tkn_indice]
      token_pos =  get_token_pos(mapper, clean_pos.start, clean_pos.stop)

      #check similarity

      if ''.join(dev_tokens[tkn_indice][token_pos]).replace('##', '') != ''.join(dev_text_data[indice][clean_pos].split()):
        print('here?!', indice,tkn_indice, special_entity, dev_tokens[tkn_indice][token_pos], '\t', token_pos, '\t', slice(deb_str,fin_str),  clean_pos )

      dev_tokens_labels[tkn_indice][token_pos.start] = 'B-'+label

      for x in range(token_pos.start+1, token_pos.stop):
        if x <len(dev_tokens_labels[tkn_indice]):
          dev_tokens_labels[tkn_indice][x] = 'I-'+label

        else:
          if x!=512 :
            print('not simillar,  \t', 'indice: \t', indice, '\t x: \t', x )


here?! 17 20 High Court For The State Of Telangana
 ['High', 'Court', 'For', 'The', 'State', 'Of', 'Telangana', 'At'] 	 slice(3, 11, None) 	 slice(7, 45, None) slice(7, 45, None)


## Datasets creation
labels_list, encode: int_labels, and dataset generation

### labels lists

#### encode labels (str -> int)

In [ ]:
ne_distribution = {}
## objective: {'ne' : num_occ}
for labels in tokens_labels:
  for idx in range(len(labels)):
    if labels[idx] in ne_distribution.keys():
      ne_distribution[labels[idx]] +=1
    else:
      ne_distribution[labels[idx]] = 1

for labels in dev_tokens_labels:
  for idx in range(len(labels)):
    if labels[idx] in ne_distribution.keys():
      ne_distribution[labels[idx]] +=1
    else:
      ne_distribution[labels[idx]] = 1


print(len(ne_distribution ))
import pandas as pd

pd.DataFrame.from_dict(dict(sorted(ne_distribution.items(), key = lambda x: x[1], reverse=True)),  orient='index').head(3)

40


,0
O,3678669
I-PRECEDENT,24789
I-CASE_NUMBER,9546


In [ ]:
labels_list = list(ne_distribution.keys())
len(labels_list)

40

In [ ]:
int_labels = {}
# for elt in labels_list:
for i, lab in enumerate(labels_list):
  int_labels[lab] = i
print(int_labels)

{'O': 0, 'B-ORG': 1, 'I-ORG': 2, 'B-OTHER_PERSON': 3, 'I-OTHER_PERSON': 4, 'B-WITNESS': 5, 'I-WITNESS': 6, 'GPE': 7, 'B-STATUTE': 8, 'I-STATUTE': 9, 'B-DATE': 10, 'I-DATE': 11, 'B-PROVISION': 12, 'I-PROVISION': 13, 'B-COURT': 14, 'I-COURT': 15, 'B-PRECEDENT': 16, 'I-PRECEDENT': 17, 'B-GPE': 18, 'I-GPE': 19, 'B-CASE_NUMBER': 20, 'I-CASE_NUMBER': 21, 'ORG': 22, 'B-PETITIONER': 23, 'I-PETITIONER': 24, 'B-JUDGE': 25, 'I-JUDGE': 26, 'WITNESS': 27, 'B-RESPONDENT': 28, 'I-RESPONDENT': 29, 'STATUTE': 30, 'RESPONDENT': 31, 'OTHER_PERSON': 32, 'DATE': 33, 'JUDGE': 34, 'PETITIONER': 35, 'PROVISION': 36, 'CASE_NUMBER': 37, 'B-LAWYER': 38, 'I-LAWYER': 39}


#### int train labels

In [ ]:
import copy
int_tokens_labels = copy.deepcopy(tokens_labels)

for i in range(len(tokens_labels)):
  for j in range(len(tokens_labels[i])):
    int_tokens_labels[i][j] = int_labels[tokens_labels[i][j]]

#### int dev labels list

In [ ]:
import copy
int_dev_tokens_labels = copy.deepcopy(dev_tokens_labels)

for i in range(len(dev_tokens_labels)):
  for j in range(len(dev_tokens_labels[i])):
    int_dev_tokens_labels[i][j] = int_labels[dev_tokens_labels[i][j]]

### generate datasets

#### gen train dt with_annot_indices

In [ ]:
def no_ne_gen():
  i = 0
  for indice in range(len(with_annot_indices)):
    yield {'labels' : int_tokens_labels[i], 'input_ids': tokenized['input_ids'][i] , 'attention_mask': tokenized['attention_mask'][i]}
    i+=1

from datasets import Dataset
with_ne_train_dt = Dataset.from_generator(no_ne_gen)
with_ne_train_dt

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 7258
})

#### gen dev dt

In [ ]:
def dev_gen():
  i = 0
  for line in dev_data: #   'id':i,  'tokens': dev_tokens[i],
    yield {'labels' : int_dev_tokens_labels[i], 'input_ids': dev_tokenized['input_ids'][i] , 'attention_mask': dev_tokenized['attention_mask'][i]}
    i+=1

# from datasets import Dataset
dev_dt = Dataset.from_generator(dev_gen)
dev_dt

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 125
})

#### display function

In [ ]:
from datasets import ClassLabel, Sequence
import pandas as pd
from IPython.display import display, HTML

def display_dt(dataset, num_examples=10):
    assert num_examples <= len(dataset);

    df = pd.DataFrame(dataset[:num_examples])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))
# show_random_elements(mapped_dataset)

In [ ]:
with_ne_train_dt_no_100 = with_ne_train_dt
dev_dt_no_100 = dev_dt

In [ ]:
display_dt(with_ne_train_dt_no_100, 2)

,labels,input_ids,attention_mask
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[101, 113, 128, 114, 1212, 2747, 15027, 1616, 1118, 1103, 3096, 1732, 1164, 1126, 3990, 1104, 19003, 119, 122, 117, 1955, 117, 3413, 117, 2260, 1113, 14304, 1334, 1104, 3475, 19890, 1403, 2950, 3300, 1104, 1134, 170, 6307, 5633, 1110, 5452, 1120, 185, 119, 1969, 1104, 15187, 3051, 112, 188, 2526, 1520, 117, 3560, 21812, 4702, 7402, 1115, 1122, 1108, 2272, 1106, 4891, 1121, 24535, 117, 16890, 24287, 111, 3291, 119, 1113, 1103, 3142, 1104, 1117, 13455, 170, 3238, 4551, 1110, 1508, 1118, 1366, 1113, 1115, 6307, 5633, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
1,"[0, 0, 0, 0, 0, 0, 3, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[101, 1124, 1108, 1145, 1455, 2480, 138, 4873, 1161, 133, 8492, 1705, 134, 107, 4610, 168, 3087, 107, 25021, 134, 107, 8492, 168, 126, 107, 135, 15531, 1592, 1302, 119, 2724, 1545, 118, 24044, 1104, 1772, 127, 133, 120, 8492, 135, 14812, 2149, 117, 1534, 118, 1107, 118, 1644, 1104, 1103, 10281, 2077, 10380, 1121, 22515, 17670, 9962, 1389, 5329, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"


In [ ]:
display_dt(dev_dt_no_100, 2)

,labels,input_ids,attention_mask
0,"[0, 0, 0, 0, 0, 0, 0, 14, 15, 15, 15, 15, 15, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 23, 24, 24, 24, 24, 0, 0, 0, 0, 0, 0, 0, 38, 39, 39, 39, 39, 0, 0, 0, 38, 39, 39, 39, 39, 39, 39, 0, 0, 0, 38, 39, 39, 0, 0, 0, 38, 39, 39, 39, 39, 39, 39, 0, ...]","[101, 109, 199, 1969, 115, 1130, 1109, 1693, 2031, 2096, 6175, 1335, 1203, 6175, 110, 13063, 10517, 1113, 131, 1955, 119, 5004, 119, 10351, 116, 6603, 119, 138, 8661, 119, 5311, 1545, 120, 1857, 111, 140, 1306, 1302, 1116, 119, 3993, 11964, 1477, 120, 1857, 117, 15722, 25631, 120, 10351, 117, 3236, 16382, 1571, 120, 10351, 13078, 11037, 3291, 4492, 119, 138, 24756, 9180, 4737, 131, 1828, 119, 156, 119, 153, 119, 18489, 117, 1828, 119, 15619, 5443, 6583, 144, 2312, 1830, 14518, 117, 1828, 119, 11896, 1197, 5329, 1105, 1828, 119, 153, 13148, 6610, 5329, 14812, 1179, 11487, 117, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]"
1,"[0, 0, 0, 0, 0, 0, 14, 15, 15, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 23, 24, 24, 24, 24, 24, 24, 24, 24, 24, 0, 0, 0, 0, 0, 0, 0, 28, 29, 29, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[101, 122, 7178, 1895, 1130, 1109, 3732, 2031, 2096, 1726, 3145, 138, 24756, 8052, 23915, 4889, 15906, 3145, 13969, 1302, 119, 3102, 1545, 2096, 17881, 1475, 113, 10789, 4253, 3929, 2096, 156, 1233, 1643, 113, 140, 114, 1302, 119, 26409, 11049, 2096, 1410, 114, 20967, 1197, 2687, 7418, 1394, 1324, 12189, 15513, 1394, 1324, 795, 138, 24756, 9180, 159, 1116, 119, 1426, 2096, 15019, 111, 2926, 1116, 119, 795, 11336, 20080, 16838, 9857, 1556, 3145, 13969, 1302, 119, 3102, 1559, 2096, 17881, 1475, 113, 10789, 4253, 3929, 2096, 156, 1233, 1643, 113, 140, 114, 1302, 119, 27724, 19297, 2096, 1410, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]"


#### affect -100 to cls, sep and pad

In [ ]:
def affect_minus_100(dt):
  #the role of this function is to find the indices of 101, 102 and 0 (cls, sep and pad) from input_ids and in the labels column affect -100
  #without having -100 in the labels list
  input_ids = dt['input_ids']
  labels = dt['labels']

  new_labs = labels.copy()

  for idx, input_id in enumerate(input_ids):
    if input_id in [101, 102, 0]:
      new_labs[idx] = -100

  return {'labels': new_labs}



In [ ]:
display_dt(dev_dt_with_100, 2)

,labels,input_ids,attention_mask
0,"[-100, 0, 0, 0, 0, 0, 0, 14, 15, 15, 15, 15, 15, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 23, 24, 24, 24, 24, 0, 0, 0, 0, 0, 0, 0, 38, 39, 39, 39, 39, 0, 0, 0, 38, 39, 39, 39, 39, 39, 39, 0, 0, 0, 38, 39, 39, 0, 0, 0, 38, 39, 39, 39, 39, 39, 39, 0, ...]","[101, 109, 199, 1969, 115, 1130, 1109, 1693, 2031, 2096, 6175, 1335, 1203, 6175, 110, 13063, 10517, 1113, 131, 1955, 119, 5004, 119, 10351, 116, 6603, 119, 138, 8661, 119, 5311, 1545, 120, 1857, 111, 140, 1306, 1302, 1116, 119, 3993, 11964, 1477, 120, 1857, 117, 15722, 25631, 120, 10351, 117, 3236, 16382, 1571, 120, 10351, 13078, 11037, 3291, 4492, 119, 138, 24756, 9180, 4737, 131, 1828, 119, 156, 119, 153, 119, 18489, 117, 1828, 119, 15619, 5443, 6583, 144, 2312, 1830, 14518, 117, 1828, 119, 11896, 1197, 5329, 1105, 1828, 119, 153, 13148, 6610, 5329, 14812, 1179, 11487, 117, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]"
1,"[-100, 0, 0, 0, 0, 0, 14, 15, 15, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 23, 24, 24, 24, 24, 24, 24, 24, 24, 24, 0, 0, 0, 0, 0, 0, 0, 28, 29, 29, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[101, 122, 7178, 1895, 1130, 1109, 3732, 2031, 2096, 1726, 3145, 138, 24756, 8052, 23915, 4889, 15906, 3145, 13969, 1302, 119, 3102, 1545, 2096, 17881, 1475, 113, 10789, 4253, 3929, 2096, 156, 1233, 1643, 113, 140, 114, 1302, 119, 26409, 11049, 2096, 1410, 114, 20967, 1197, 2687, 7418, 1394, 1324, 12189, 15513, 1394, 1324, 795, 138, 24756, 9180, 159, 1116, 119, 1426, 2096, 15019, 111, 2926, 1116, 119, 795, 11336, 20080, 16838, 9857, 1556, 3145, 13969, 1302, 119, 3102, 1559, 2096, 17881, 1475, 113, 10789, 4253, 3929, 2096, 156, 1233, 1643, 113, 140, 114, 1302, 119, 27724, 19297, 2096, 1410, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]"


#### save datasets

In [ ]:
# O, cls, sep and pad labelled as 0
with_ne_train_dt.save_to_disk('/content/drive/MyDrive/ner_india/with_ne_train_dtst_no_100')
dev_dt.save_to_disk('/content/drive/MyDrive/ner_india/dev_dtst_no_100')


# O, cls, sep and pad labelled as -100
with_ne_train_dt.map(affect_minus_100).save_to_disk('/content/drive/MyDrive/ner_india/with_ne_train_dtst_with-100')
dev_dt.map(affect_minus_100).save_to_disk('/content/drive/MyDrive/ner_india/dev_dtst_with-100')


In [ ]:
with_ne_train_dt_with_100 = with_ne_train_dt.map(affect_minus_100)
dev_dt_with_100 = dev_dt.map(affect_minus_100)

Map:   0%|          | 0/7258 [00:00<?, ? examples/s]

Map:   0%|          | 0/125 [00:00<?, ? examples/s]

# 3.Process

#### load datasets

##### display function

In [ ]:
from datasets import ClassLabel, Sequence
import pandas as pd
from IPython.display import display, HTML

def display_dt(dataset, num_examples=10):
    assert num_examples <= len(dataset);

    df = pd.DataFrame(dataset[:num_examples])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))
# show_random_elements(mapped_dataset)

In [ ]:
from datasets import load_from_disk
with_ne_train_dtst_with_100 = load_from_disk('/content/drive/MyDrive/ner_india/with_ne_train_dtst_with-100')
display_dt(with_ne_train_dtst_with_100, 2)

,labels,input_ids,attention_mask
0,"[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, ...]","[101, 113, 128, 114, 1212, 2747, 15027, 1616, 1118, 1103, 3096, 1732, 1164, 1126, 3990, 1104, 19003, 119, 122, 117, 1955, 117, 3413, 117, 2260, 1113, 14304, 1334, 1104, 3475, 19890, 1403, 2950, 3300, 1104, 1134, 170, 6307, 5633, 1110, 5452, 1120, 185, 119, 1969, 1104, 15187, 3051, 112, 188, 2526, 1520, 117, 3560, 21812, 4702, 7402, 1115, 1122, 1108, 2272, 1106, 4891, 1121, 24535, 117, 16890, 24287, 111, 3291, 119, 1113, 1103, 3142, 1104, 1117, 13455, 170, 3238, 4551, 1110, 1508, 1118, 1366, 1113, 1115, 6307, 5633, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
1,"[-100, 0, 0, 0, 0, 0, 3, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 4, 4, 0, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, ...]","[101, 1124, 1108, 1145, 1455, 2480, 138, 4873, 1161, 133, 8492, 1705, 134, 107, 4610, 168, 3087, 107, 25021, 134, 107, 8492, 168, 126, 107, 135, 15531, 1592, 1302, 119, 2724, 1545, 118, 24044, 1104, 1772, 127, 133, 120, 8492, 135, 14812, 2149, 117, 1534, 118, 1107, 118, 1644, 1104, 1103, 10281, 2077, 10380, 1121, 22515, 17670, 9962, 1389, 5329, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"


In [ ]:
from datasets import load_from_disk
dev_dtst_with_100 = load_from_disk('/content/drive/MyDrive/ner_india/dev_dtst_with-100')
display_dt(dev_dtst_with_100, 2)

,labels,input_ids,attention_mask
0,"[-100, 0, 0, 0, 0, 0, 0, 14, 15, 15, 15, 15, 15, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 23, 24, 24, 24, 24, 0, 0, 0, 0, 0, 0, 0, 38, 39, 39, 39, 39, 0, 0, 0, 38, 39, 39, 39, 39, 39, 39, 0, 0, 0, 38, 39, 39, 0, 0, 0, 38, 39, 39, 39, 39, 39, 39, 0, ...]","[101, 109, 199, 1969, 115, 1130, 1109, 1693, 2031, 2096, 6175, 1335, 1203, 6175, 110, 13063, 10517, 1113, 131, 1955, 119, 5004, 119, 10351, 116, 6603, 119, 138, 8661, 119, 5311, 1545, 120, 1857, 111, 140, 1306, 1302, 1116, 119, 3993, 11964, 1477, 120, 1857, 117, 15722, 25631, 120, 10351, 117, 3236, 16382, 1571, 120, 10351, 13078, 11037, 3291, 4492, 119, 138, 24756, 9180, 4737, 131, 1828, 119, 156, 119, 153, 119, 18489, 117, 1828, 119, 15619, 5443, 6583, 144, 2312, 1830, 14518, 117, 1828, 119, 11896, 1197, 5329, 1105, 1828, 119, 153, 13148, 6610, 5329, 14812, 1179, 11487, 117, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]"
1,"[-100, 0, 0, 0, 0, 0, 14, 15, 15, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 23, 24, 24, 24, 24, 24, 24, 24, 24, 24, 0, 0, 0, 0, 0, 0, 0, 28, 29, 29, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[101, 122, 7178, 1895, 1130, 1109, 3732, 2031, 2096, 1726, 3145, 138, 24756, 8052, 23915, 4889, 15906, 3145, 13969, 1302, 119, 3102, 1545, 2096, 17881, 1475, 113, 10789, 4253, 3929, 2096, 156, 1233, 1643, 113, 140, 114, 1302, 119, 26409, 11049, 2096, 1410, 114, 20967, 1197, 2687, 7418, 1394, 1324, 12189, 15513, 1394, 1324, 795, 138, 24756, 9180, 159, 1116, 119, 1426, 2096, 15019, 111, 2926, 1116, 119, 795, 11336, 20080, 16838, 9857, 1556, 3145, 13969, 1302, 119, 3102, 1559, 2096, 17881, 1475, 113, 10789, 4253, 3929, 2096, 156, 1233, 1643, 113, 140, 114, 1302, 119, 27724, 19297, 2096, 1410, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]"


##### labels_list

In [ ]:
labels_list = ['O', 'B-ORG', 'I-ORG', 'B-OTHER_PERSON', 'I-OTHER_PERSON', 'B-WITNESS', 'I-WITNESS', 'GPE', 'B-STATUTE', 'I-STATUTE', 'B-DATE', 'I-DATE', 'B-PROVISION', 'I-PROVISION', 'B-COURT', 'I-COURT', 'B-PRECEDENT', 'I-PRECEDENT', 'B-GPE', 'I-GPE', 'B-CASE_NUMBER', 'I-CASE_NUMBER', 'ORG', 'B-PETITIONER', 'I-PETITIONER', 'B-JUDGE', 'I-JUDGE', 'WITNESS', 'B-RESPONDENT', 'I-RESPONDENT', 'STATUTE', 'RESPONDENT', 'OTHER_PERSON', 'DATE', 'JUDGE', 'PETITIONER', 'PROVISION', 'CASE_NUMBER', 'B-LAWYER', 'I-LAWYER']

int_labels = {}
for i, lab in enumerate(labels_list):
  int_labels[lab] = i

print(int_labels)

{'O': 0, 'B-ORG': 1, 'I-ORG': 2, 'B-OTHER_PERSON': 3, 'I-OTHER_PERSON': 4, 'B-WITNESS': 5, 'I-WITNESS': 6, 'GPE': 7, 'B-STATUTE': 8, 'I-STATUTE': 9, 'B-DATE': 10, 'I-DATE': 11, 'B-PROVISION': 12, 'I-PROVISION': 13, 'B-COURT': 14, 'I-COURT': 15, 'B-PRECEDENT': 16, 'I-PRECEDENT': 17, 'B-GPE': 18, 'I-GPE': 19, 'B-CASE_NUMBER': 20, 'I-CASE_NUMBER': 21, 'ORG': 22, 'B-PETITIONER': 23, 'I-PETITIONER': 24, 'B-JUDGE': 25, 'I-JUDGE': 26, 'WITNESS': 27, 'B-RESPONDENT': 28, 'I-RESPONDENT': 29, 'STATUTE': 30, 'RESPONDENT': 31, 'OTHER_PERSON': 32, 'DATE': 33, 'JUDGE': 34, 'PETITIONER': 35, 'PROVISION': 36, 'CASE_NUMBER': 37, 'B-LAWYER': 38, 'I-LAWYER': 39}


In [ ]:
tokenizer

BertTokenizerFast(name_or_path='dslim/bert-base-NER', vocab_size=28996, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [ ]:
model

'dslim/bert-base-NER'

In [ ]:
import transformers
from transformers import  DataCollatorForTokenClassification, AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification, pipeline

In [ ]:
# Load Sqeval.
import evaluate

metric = evaluate.load("seqeval")

# Create the list with the tags.

# Function to compute precision, recall, F1 and accuracy.
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [labels_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [labels_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

#### model

In [ ]:
# Load Sqeval.
import evaluate
import numpy as np

metric = evaluate.load("seqeval")

# Create the list with the tags.

# Function to compute precision, recall, F1 and accuracy.
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [labels_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [labels_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

#### train on created datasets

In [ ]:
model_checkpoint = 'dslim/bert-base-NER'
batch_size = 16

model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    model_name,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
    remove_unused_columns=False
)

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(labels_list), ignore_mismatched_sizes=True)


data_collator = DataCollatorForTokenClassification(tokenizer)

trainer = Trainer(
    model,
    args,
    train_dataset= with_ne_train_dt_with_100,
    eval_dataset= dev_dt_with_100,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at dslim/bert-base-NE

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,1.979785,0.062182,0.142091,0.086507,0.554392
2,0.352500,2.026945,0.069888,0.156390,0.096605,0.582652
3,0.134600,2.195241,0.074827,0.164433,0.102851,0.578243


/content/drive/MyDrive/virtual_env/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: GPE seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/content/drive/MyDrive/virtual_env/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/content/drive/MyDrive/virtual_env/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/content/drive/MyDrive/virtual_env/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: GPE seems not to be NE tag.
  warnings.w

TrainOutput(global_step=1362, training_loss=0.204241961284531, metrics={'train_runtime': 1993.8901, 'train_samples_per_second': 10.92, 'train_steps_per_second': 0.683, 'total_flos': 5691430232801280.0, 'train_loss': 0.204241961284531, 'epoch': 3.0})

##### train on datasets without -100

In [ ]:
display_dt(with_ne_train_dt, 1)

,labels,input_ids,attention_mask
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[101, 113, 128, 114, 1212, 2747, 15027, 1616, 1118, 1103, 3096, 1732, 1164, 1126, 3990, 1104, 19003, 119, 122, 117, 1955, 117, 3413, 117, 2260, 1113, 14304, 1334, 1104, 3475, 19890, 1403, 2950, 3300, 1104, 1134, 170, 6307, 5633, 1110, 5452, 1120, 185, 119, 1969, 1104, 15187, 3051, 112, 188, 2526, 1520, 117, 3560, 21812, 4702, 7402, 1115, 1122, 1108, 2272, 1106, 4891, 1121, 24535, 117, 16890, 24287, 111, 3291, 119, 1113, 1103, 3142, 1104, 1117, 13455, 170, 3238, 4551, 1110, 1508, 1118, 1366, 1113, 1115, 6307, 5633, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"


In [ ]:
display_dt(dev_dt, 1)

,labels,input_ids,attention_mask
0,"[0, 0, 0, 0, 0, 0, 0, 14, 15, 15, 15, 15, 15, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 23, 24, 24, 24, 24, 0, 0, 0, 0, 0, 0, 0, 38, 39, 39, 39, 39, 0, 0, 0, 38, 39, 39, 39, 39, 39, 39, 0, 0, 0, 38, 39, 39, 0, 0, 0, 38, 39, 39, 39, 39, 39, 39, 0, ...]","[101, 109, 199, 1969, 115, 1130, 1109, 1693, 2031, 2096, 6175, 1335, 1203, 6175, 110, 13063, 10517, 1113, 131, 1955, 119, 5004, 119, 10351, 116, 6603, 119, 138, 8661, 119, 5311, 1545, 120, 1857, 111, 140, 1306, 1302, 1116, 119, 3993, 11964, 1477, 120, 1857, 117, 15722, 25631, 120, 10351, 117, 3236, 16382, 1571, 120, 10351, 13078, 11037, 3291, 4492, 119, 138, 24756, 9180, 4737, 131, 1828, 119, 156, 119, 153, 119, 18489, 117, 1828, 119, 15619, 5443, 6583, 144, 2312, 1830, 14518, 117, 1828, 119, 11896, 1197, 5329, 1105, 1828, 119, 153, 13148, 6610, 5329, 14812, 1179, 11487, 117, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]"


In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset= with_ne_train_dt,  #without -100
    eval_dataset= dev_dt,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,1.058191,0.081013,0.171582,0.110060,0.787469
2,0.012700,1.061291,0.071856,0.160858,0.099338,0.800188
3,0.005100,1.182585,0.075868,0.168007,0.104532,0.786109


/content/drive/MyDrive/virtual_env/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: GPE seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/content/drive/MyDrive/virtual_env/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/content/drive/MyDrive/virtual_env/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/content/drive/MyDrive/virtual_env/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: GPE seems not to be NE tag.
  warnings.w

TrainOutput(global_step=1362, training_loss=0.00772375130968472, metrics={'train_runtime': 2107.2855, 'train_samples_per_second': 10.333, 'train_steps_per_second': 0.646, 'total_flos': 5691430232801280.0, 'train_loss': 0.00772375130968472, 'epoch': 3.0})

#### train on loaded datasets

##### with -100

In [ ]:
from datasets import load_from_disk

with_ne_train_dtst_with_100 = load_from_disk('/content/drive/MyDrive/ner_india/with_ne_train_dtst_with-100')
dev_dtst_with_100 = load_from_disk('/content/drive/MyDrive/ner_india/dev_dtst_with-100')


In [ ]:
model_checkpoint = 'dslim/bert-base-NER'
batch_size = 16

model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    model_name,
    eval_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
    remove_unused_columns=False
)

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(labels_list), ignore_mismatched_sizes=True)


data_collator = DataCollatorForTokenClassification(tokenizer)

trainer = Trainer(
    model,
    args,
    train_dataset= with_ne_train_dt_with_100,
    eval_dataset= dev_dt_with_100,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at dslim/bert-base-NER and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([40]) in the model instantiated
- classifier.weight: found shape torch.Size([9, 768])

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,1.913024,0.061827,0.138517,0.085494,0.569425
2,0.358400,2.040764,0.070224,0.159964,0.097601,0.573069
3,0.131900,2.176579,0.072043,0.157283,0.098821,0.577968


/content/drive/MyDrive/virtual_env/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: GPE seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/content/drive/MyDrive/virtual_env/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/content/drive/MyDrive/virtual_env/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/content/drive/MyDrive/virtual_env/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: GPE seems not to be NE tag.
  warnings.w

TrainOutput(global_step=1362, training_loss=0.2050603230794271, metrics={'train_runtime': 1997.2391, 'train_samples_per_second': 10.902, 'train_steps_per_second': 0.682, 'total_flos': 5691430232801280.0, 'train_loss': 0.2050603230794271, 'epoch': 3.0})

#### train without -100 in int labels

In [ ]:
with_ne_train_dtst_without_100 = load_from_disk('/content/drive/MyDrive/ner_india/with_ne_train_dtst_no_100')
dev_dtst_without_100 = load_from_disk('/content/drive/MyDrive/ner_india/dev_dtst_no_100')

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset= with_ne_train_dtst_without_100,
    eval_dataset= dev_dtst_without_100,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,1.384539,0.068261,0.158177,0.095366,0.776656
2,0.003200,1.493755,0.072276,0.174263,0.102174,0.770266
3,0.002400,1.420731,0.079513,0.175156,0.109375,0.786641


/content/drive/MyDrive/virtual_env/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: GPE seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/content/drive/MyDrive/virtual_env/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OTHER_PERSON seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/content/drive/MyDrive/virtual_env/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/content/drive/MyDrive/virtual_env/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior

TrainOutput(global_step=1362, training_loss=0.0026925060017343487, metrics={'train_runtime': 2109.9367, 'train_samples_per_second': 10.32, 'train_steps_per_second': 0.646, 'total_flos': 5691430232801280.0, 'train_loss': 0.0026925060017343487, 'epoch': 3.0})

### Get rid of the labels that doesn't follow the BIO format